In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
# -------- TEST USER ----------- #

TEST_USER      = '001'

BASE_DIR       = '../'
IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 7
LEARNING_RATE  = 0.001

In [3]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

In [4]:
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

loading data for j gesture on the XY plane ... √
loading data for z gesture on the XY plane ... √
loading data for bad gesture on the XY plane ... √
loading data for deaf gesture on the XY plane ... √
loading data for fine gesture on the XY plane ... √
loading data for good gesture on the XY plane ... √
loading data for goodbye gesture on the XY plane ... √
loading data for hello gesture on the XY plane ... √
loading data for hungry gesture on the XY plane ... √
loading data for me gesture on the XY plane ... √
loading data for no gesture on the XY plane ... √
loading data for please gesture on the XY plane ... √
loading data for sorry gesture on the XY plane ... √
loading data for thankyou gesture on the XY plane ... √
loading data for yes gesture on the XY plane ... √
loading data for you gesture on the XY plane ... √
loading data for j gesture on the YZ plane ... √
loading data for z gesture on the YZ plane ... √
loading data for bad gesture on the YZ plane ... √
loading data for de

In [5]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [6]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(GESTURES))

In [8]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [9]:
model_xy = get_model()
history_xy = model_xy.fit(X_train_xy, y_train_xy, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 12s 337ms/step - loss: 2.8433 - accuracy: 0.1527
Epoch 2/7
30/30 [==============================] - 10s 338ms/step - loss: 1.6849 - accuracy: 0.4650
Epoch 3/7
30/30 [==============================] - 10s 332ms/step - loss: 1.2545 - accuracy: 0.5974
Epoch 4/7
30/30 [==============================] - 10s 333ms/step - loss: 1.0453 - accuracy: 0.6714
Epoch 5/7
30/30 [==============================] - 10s 330ms/step - loss: 0.8573 - accuracy: 0.7162
Epoch 6/7
30/30 [==============================] - 10s 332ms/step - loss: 0.7341 - accuracy: 0.7755
Epoch 7/7
30/30 [==============================] - 10s 331ms/step - loss: 0.6628 - accuracy: 0.8088


In [10]:
# prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
# y_pred_xy = prob_xy.predict(X_test_xy)
y_pred_xy = model_xy.predict(X_test_xy)
y_pred = np.argmax(y_pred_xy, axis=1)
print(classification_report(y_test_xy.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.90      0.95        10
         1.0       0.00      0.00      0.00        10
         2.0       0.88      0.70      0.78        10
         3.0       1.00      0.80      0.89        10
         4.0       1.00      1.00      1.00        10
         5.0       0.56      0.50      0.53        10
         6.0       1.00      1.00      1.00        10
         7.0       0.45      0.90      0.60        10
         8.0       1.00      0.80      0.89        10
         9.0       0.50      0.70      0.58        10
        10.0       0.91      1.00      0.95        10
        11.0       0.53      0.90      0.67        10
        12.0       0.67      0.20      0.31        10
        13.0       1.00      0.90      0.95        10
        14.0       0.91      1.00      0.95        10
        15.0       0.82      0.90      0.86        10

    accuracy                           0.76       160
   macro avg       0.76   

In [11]:
model_yz = get_model()
history_yz = model_yz.fit(X_train_yz, y_train_yz, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 12s 333ms/step - loss: 3.4304 - accuracy: 0.1674
Epoch 2/7
30/30 [==============================] - 10s 330ms/step - loss: 1.9345 - accuracy: 0.3785
Epoch 3/7
30/30 [==============================] - 10s 329ms/step - loss: 1.2123 - accuracy: 0.5925
Epoch 4/7
30/30 [==============================] - 10s 331ms/step - loss: 0.9359 - accuracy: 0.7039
Epoch 5/7
30/30 [==============================] - 10s 332ms/step - loss: 0.7506 - accuracy: 0.7750
Epoch 6/7
30/30 [==============================] - 10s 331ms/step - loss: 0.6695 - accuracy: 0.7951
Epoch 7/7
30/30 [==============================] - 10s 333ms/step - loss: 0.5708 - accuracy: 0.8372


In [12]:
# prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
# y_pred_yz = prob_yz.predict(X_test_yz)
y_pred_yz = model_yz.predict(X_test_yz)
y_pred = np.argmax(y_pred_yz, axis=1)
print(classification_report(y_test_yz.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        10
         1.0       0.50      0.20      0.29        10
         2.0       0.33      0.20      0.25        10
         3.0       0.59      1.00      0.74        10
         4.0       0.56      0.90      0.69        10
         5.0       0.71      0.50      0.59        10
         6.0       0.86      0.60      0.71        10
         7.0       0.54      0.70      0.61        10
         8.0       0.69      0.90      0.78        10
         9.0       1.00      1.00      1.00        10
        10.0       0.71      1.00      0.83        10
        11.0       0.64      0.70      0.67        10
        12.0       0.44      0.40      0.42        10
        13.0       0.90      0.90      0.90        10
        14.0       0.30      0.30      0.30        10
        15.0       0.50      0.60      0.55        10

    accuracy                           0.62       160
   macro avg       0.58   

In [13]:
model_zx = get_model()
history_zx = model_zx.fit(X_train_zx, y_train_zx, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 12s 330ms/step - loss: 3.4220 - accuracy: 0.2079
Epoch 2/7
30/30 [==============================] - 10s 328ms/step - loss: 2.2226 - accuracy: 0.3465
Epoch 3/7
30/30 [==============================] - 10s 332ms/step - loss: 1.6950 - accuracy: 0.4751
Epoch 4/7
30/30 [==============================] - 10s 332ms/step - loss: 1.3947 - accuracy: 0.5541
Epoch 5/7
30/30 [==============================] - 10s 331ms/step - loss: 1.0826 - accuracy: 0.6571
Epoch 6/7
30/30 [==============================] - 10s 331ms/step - loss: 0.9932 - accuracy: 0.7113
Epoch 7/7
30/30 [==============================] - 10s 331ms/step - loss: 0.8431 - accuracy: 0.7623


In [14]:
# prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
# y_pred_zx = prob_zx.predict(X_test_zx)
y_pred_zx = model_zx.predict(X_test_zx)
y_pred = np.argmax(y_pred_zx, axis=1)
print(classification_report(y_test_zx.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.42      0.50      0.45        10
         1.0       0.45      0.90      0.60        10
         2.0       0.14      0.10      0.12        10
         3.0       0.78      0.70      0.74        10
         4.0       0.00      0.00      0.00        10
         5.0       0.77      1.00      0.87        10
         6.0       0.56      1.00      0.71        10
         7.0       0.00      0.00      0.00        10
         8.0       1.00      0.90      0.95        10
         9.0       0.60      0.30      0.40        10
        10.0       0.43      0.90      0.58        10
        11.0       0.17      0.40      0.24        10
        12.0       1.00      0.20      0.33        10
        13.0       0.62      0.80      0.70        10
        14.0       0.00      0.00      0.00        10
        15.0       0.67      0.40      0.50        10

    accuracy                           0.51       160
   macro avg       0.47   

/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
y_total = y_pred_xy + y_pred_yz + y_pred_zx
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      0.40      0.57        10
         2.0       1.00      0.90      0.95        10
         3.0       1.00      1.00      1.00        10
         4.0       0.91      1.00      0.95        10
         5.0       1.00      1.00      1.00        10
         6.0       1.00      1.00      1.00        10
         7.0       0.67      1.00      0.80        10
         8.0       1.00      1.00      1.00        10
         9.0       1.00      1.00      1.00        10
        10.0       1.00      1.00      1.00        10
        11.0       0.42      1.00      0.59        10
        12.0       1.00      0.10      0.18        10
        13.0       1.00      1.00      1.00        10
        14.0       0.80      0.40      0.53        10
        15.0       0.91      1.00      0.95        10

    accuracy                           0.86       160
   macro avg       0.92   

In [16]:
config = '\n\nTEST_USER ' + TEST_USER + '\n'
underline = '=====================================\n'
log_dir = os.path.join(BASE_DIR, LOG_DIR)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
f = open(os.path.join(log_dir, 'logs_sptl_bw.txt'), 'a')
f.write(config)
f.write(underline)
f.write(report)
f.close()